# Capstone Two Pre-processing & Training Data Development

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

from library.sb_utils import save_file


In [2]:
realtor_data = pd.read_csv('realtor-data.csv')
realtor_data.shape

(1401066, 10)

In [3]:
# drop rows with missing value in the city,price and zip_colde columns

realtor_data2_drop=realtor_data
realtor_data2_drop.dropna(subset =['zip_code','price','city'],inplace=True)
missing = pd.concat([realtor_data2_drop.isnull().sum(), 100 * realtor_data2_drop.isnull().mean()], axis=1)
missing.columns=['count','%']
missing.sort_values(by='%')

,count,%
status,0,0.000000
city,0,0.000000
state,0,0.000000
zip_code,0,0.000000
price,0,0.000000
bath,193804,13.838932
bed,216158,15.435160
acre_lot,357319,25.515022
house_size,449764,32.116227
prev_sold_date,685717,48.964886


In [5]:
# Check data types of columns
for column in realtor_data2_drop.columns[realtor_data2_drop.isnull().sum() > 0]:
    if realtor_data2_drop[column].dtype != 'object':  # Check if column is numeric
        mean_val = realtor_data2_drop[column].mean()
        realtor_data2_drop[column].fillna(mean_val, inplace=True)

In [6]:
missing = pd.concat([realtor_data2_drop.isnull().sum(), 100 * realtor_data2_drop.isnull().mean()], axis=1)
missing.columns=['count','%']
missing.sort_values(by='%')

,count,%
status,0,0.000000
bed,0,0.000000
bath,0,0.000000
acre_lot,0,0.000000
city,0,0.000000
state,0,0.000000
zip_code,0,0.000000
house_size,0,0.000000
price,0,0.000000
prev_sold_date,685717,48.964886


In [7]:
# fill missing value with mean
  
for column in realtor_data2_drop.columns[realtor_data2_drop.isnull().sum() > 0]:
    if realtor_data2_drop[column].dtype != 'object':  # Check if column is numeric
        mean_val = realtor_data2_drop[column].mean()
        realtor_data2_drop[column].fillna(mean_val, inplace=True)

In [10]:
realtor_data2_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400426 entries, 0 to 1401065
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   status          1400426 non-null  object 
 1   bed             1400426 non-null  float64
 2   bath            1400426 non-null  float64
 3   acre_lot        1400426 non-null  float64
 4   city            1400426 non-null  object 
 5   state           1400426 non-null  object 
 6   zip_code        1400426 non-null  float64
 7   house_size      1400426 non-null  float64
 8   prev_sold_date  714709 non-null   object 
 9   price           1400426 non-null  float64
dtypes: float64(6), object(4)
memory usage: 117.5+ MB


Create dummy or indicator features for categorical variables

In [12]:
realtor_data2_drop.status

0          for_sale
1          for_sale
2          for_sale
3          for_sale
4          for_sale
             ...   
1401061    for_sale
1401062    for_sale
1401063    for_sale
1401064    for_sale
1401065    for_sale
Name: status, Length: 1400426, dtype: object

In [13]:
realtor_data2_drop.status.unique()

array(['for_sale', 'ready_to_build'], dtype=object)

In [15]:
# turns status column into a dummy variable 
dummy=pd.get_dummies(realtor_data2_drop['status'])
dummy.head()

,for_sale,ready_to_build
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [19]:
df=pd.concat([realtor_data2_drop,dummy],axis=1)
df.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price,for_sale,ready_to_build
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.000000,NaN,105000.0,1,0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.000000,NaN,80000.0,1,0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.000000,NaN,67000.0,1,0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.000000,NaN,145000.0,1,0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,2178.735694,NaN,65000.0,1,0


In [20]:
df=realtor_data2_drop.merge(dummy,left_index=True,right_index=True)
df.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price,for_sale,ready_to_build
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.000000,NaN,105000.0,1,0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.000000,NaN,80000.0,1,0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.000000,NaN,67000.0,1,0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.000000,NaN,145000.0,1,0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,2178.735694,NaN,65000.0,1,0


Standardize the magnitude of numeric features using a scaler

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400426 entries, 0 to 1401065
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   status          1400426 non-null  object 
 1   bed             1400426 non-null  float64
 2   bath            1400426 non-null  float64
 3   acre_lot        1400426 non-null  float64
 4   city            1400426 non-null  object 
 5   state           1400426 non-null  object 
 6   zip_code        1400426 non-null  float64
 7   house_size      1400426 non-null  float64
 8   prev_sold_date  714709 non-null   object 
 9   price           1400426 non-null  float64
 10  for_sale        1400426 non-null  uint8  
 11  ready_to_build  1400426 non-null  uint8  
dtypes: float64(6), object(4), uint8(2)
memory usage: 152.4+ MB


In [23]:
from sklearn.preprocessing import StandardScaler

# Assuming your DataFrame is named 'df'
numeric_columns = ['bed', 'bath', 'acre_lot', 'zip_code', 'house_size', 'price']

# Select only the numeric columns for standardization
numeric_df = df[numeric_columns]

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the numeric data
scaled_features = scaler.fit_transform(numeric_df)

# Create a DataFrame with the scaled features
scaled_df = pd.DataFrame(scaled_features, columns=numeric_columns)

# Concatenate the scaled numeric features with non-numeric columns if needed
# For example:
# final_df = pd.concat([scaled_df, df[['status', 'city', 'state', 'prev_sold_date', 'for_sale', 'ready_to_build']]], axis=1)

# Display the first few rows of the scaled DataFrame
print(scaled_df.head())

        bed      bath  acre_lot  zip_code  house_size     price
0 -0.207924 -0.280304 -0.029690 -1.878555   -0.437376 -0.265985
1  0.322010 -0.280304 -0.029727 -1.878555   -0.226460 -0.275301
2 -0.737857 -0.855278 -0.029662 -1.831562   -0.497141 -0.280145
3  0.322010 -0.280304 -0.029708 -1.847065   -0.131600 -0.251079
4  1.381877 -0.280304 -0.029755 -1.859419    0.000000 -0.280891


Split your data into testing and training datasets

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming your DataFrame is named 'df' and you have already standardized the numeric columns
numeric_columns = ['bed', 'bath', 'acre_lot', 'zip_code', 'house_size', 'price']

# Select only the numeric columns for standardization
numeric_df = df[numeric_columns]

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the numeric data
scaled_features = scaler.fit_transform(numeric_df)

# Create a DataFrame with the scaled features
scaled_df = pd.DataFrame(scaled_features, columns=numeric_columns)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_df, df['price'], test_size=0.2, random_state=42)

# Print shapes to verify the split
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (1120340, 6) (1120340,)
Testing set shape: (280086, 6) (280086,)
